In [1]:
import json, base64, os, tiktoken, random, time, threading
from dataclasses import dataclass
from typing import List, Optional, Dict, Any
from dotenv import load_dotenv
from openai import AzureOpenAI
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import azure.cognitiveservices.speech as speechsdk
import requests, pygame
from pathlib import Path
import soundfile as sf, sounddevice as sd
import librosa, librosa.display, tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from io import BytesIO
from azure.storage.blob import BlobServiceClient

load_dotenv()

C:\Users\blank\AppData\Roaming\Python\Python313\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.13.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


ModuleNotFoundError: No module named 'tensorflow'

# Data Class

In [ ]:
@dataclass
class StrangeResponse:
    question: str; answer: str; timestamp: str; severity: str
    emotion: str = "중립"; answer_quality: str = "normal"

@dataclass  
class ConversationTurn:
    question: str; answer: str; timestamp: str
    emotion: str = "중립"; answer_length: int = 0
    answer_quality: str = "normal"; audio_file: str = ""

class Config:
    ENDPOINT = os.getenv("gpt-endpoint")
    DEPLOYMENT = "gpt-4o"
    SUBSCRIPTION_KEY = os.getenv("gpt-key")
    API_VERSION = "2024-02-15-preview"
    SPEECH_KEY = os.getenv("speech-key")
    SPEECH_REGION = "eastus"
    MAX_TOKENS = 4000

# 음성 분석 설정
SR, FIXED_DURATION = 16000, 30
category = {0: 'cc', 1: 'cd'}

print(f"🔧 설정: OpenAI {'✅' if Config.ENDPOINT and Config.SUBSCRIPTION_KEY else '❌'} | Speech {'✅' if Config.SPEECH_KEY else '❌'}")


# Image Analysis

In [ ]:
class ImageAnalyzer:
    def __init__(self):
        self.client = AzureOpenAI(
            api_version=Config.API_VERSION,
            azure_endpoint=Config.ENDPOINT,
            api_key=Config.SUBSCRIPTION_KEY,
        )
    
    def analyze_image(self, image_path):
        try:
            with open(image_path, "rb") as f:
                base64_image = base64.b64encode(f.read()).decode('utf-8')
                
            response = self.client.chat.completions.create(
                model=Config.DEPLOYMENT,
                messages=[{"role": "user", "content": [
                    {"type": "text", "text": """이미지를 분석해서 JSON으로 답해주세요:
{"caption": "전체 설명", "dense_captions": ["세부1", "세부2"], "mood": "분위기", 
"time_period": "시대", "key_objects": ["객체1", "객체2"], "people_description": "인물 설명",
"people_count": 숫자, "time_of_day": "시간대"}"""},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]}], max_tokens=1000, temperature=0.3
            )
            
            text = response.choices[0].message.content
            if "```json" in text:
                text = text[text.find("```json")+7:text.find("```", text.find("```json")+7)]
            elif "{" in text:
                text = text[text.find("{"):text.rfind("}")+1]
            return json.loads(text)
        except:
            return None

print("✅ ImageAnalyzer 클래스 정의 완료")

# Chat System


In [ ]:
class ChatSystem:
    def __init__(self):
        self.client = AzureOpenAI(api_version=Config.API_VERSION, azure_endpoint=Config.ENDPOINT, api_key=Config.SUBSCRIPTION_KEY)
        self.conversation_history, self.conversation_turns = [], []
        self.tokenizer = tiktoken.get_encoding("cl100k_base")
        self.token_count, self.last_question = 0, ""
        self.recording, self.audio_data, self.all_audio_data = False, [], []
        self.sample_rate = 44100
        
        # 디렉토리 설정
        self.session_id = datetime.now().strftime("%Y%m%d_%H%M%S")
        for base_dir in ["audio_records", "audio_records_combined"]:
            (Path(base_dir) / self.session_id).mkdir(parents=True, exist_ok=True)
        self.session_dir = Path("audio_records") / self.session_id
        self.session_audio_dir = Path("audio_records_combined") / self.session_id
    
    def start_recording(self):
        if self.recording: return
        self.recording, self.audio_data = True, []
        
        def callback(indata, frames, time, status):
            if self.recording: self.audio_data.append(indata.copy())
        
        self.audio_thread = sd.InputStream(samplerate=self.sample_rate, channels=1, callback=callback)
        self.audio_thread.start()
    
    def stop_recording(self):
        if not self.recording: return None
        self.recording = False
        if hasattr(self, 'audio_thread'):
            self.audio_thread.stop(); self.audio_thread.close()
        
        if self.audio_data:
            audio_data = np.concatenate(self.audio_data, axis=0)
            filename = self.session_dir / f"record_{datetime.now().strftime('%H%M%S')}.wav"
            sf.write(filename, audio_data, self.sample_rate)
            self.all_audio_data.append(audio_data)
            
            # 전체 세션 파일 업데이트
            combined = np.concatenate(self.all_audio_data, axis=0)
            sf.write(self.session_audio_dir / f"{self.session_id}.wav", combined, self.sample_rate)
            return str(filename)
        return None
    
    def setup_conversation_context(self, analysis_result):
        info = {k: analysis_result.get(k, "") for k in ["caption", "mood", "time_period", "time_of_day", "people_description"]}
        info.update({
            "dense_captions": "\n".join(f"- {dc}" for dc in analysis_result.get("dense_captions", [])),
            "key_objects": ", ".join(analysis_result.get("key_objects", [])),
            "people_count": analysis_result.get("people_count", 0)
        })
        
        system_message = f"""너는 노인과 대화하는 요양보호사야. 노인과 특정 이미지에 대해서 질의응답을 주고받아. 
노인은 치매 증상이 갑자기 나타날 수도 있어. 반복되는 말에도 똑같이 대답해줘야 해. 
친절하고 어른을 공경하는 말투여야 해. 그리고 공감을 잘 해야 해. 예의도 지켜. 
너는 주로 질문을 하는 쪽이고, 노인은 대답을 해줄거야. 대답에 대한 리액션과 함께 적절히 대화를 이어 가.

=== 이미지 정보 ===
주요 설명: {info['caption']} | 분위기: {info['mood']} | 시대: {info['time_period']}
시간대: {info['time_of_day']} | 인원: {info['people_count']}명 | 객체: {info['key_objects']}
인물: {info['people_description']}
세부: {info['dense_captions']}

=== 대화 원칙 ===
간결하게: 50자 이내로 질문하기 | 사진 주제 유지 | 심도있는 대화 | 공감하기 | 하나씩만 질문 | 따뜻하게"""
        
        self.conversation_history = [{"role": "system", "content": system_message}]
        self.token_count = len(self.tokenizer.encode(system_message))
    
    def generate_initial_question(self):
        response = self.client.chat.completions.create(
            model=Config.DEPLOYMENT,
            messages=self.conversation_history + [{"role": "user", "content": "어르신께 따듯하고 친근하게 사진에 대하여 질문을 해주세요. 50자 이내로 간결하게 질문해주세요."}],
            max_tokens=512, temperature=0.8
        )
        
        question = response.choices[0].message.content
        self.conversation_history.append({"role": "assistant", "content": question})
        self.token_count += len(self.tokenizer.encode(question))
        self.last_question = question
        return question

    def chat_about_image(self, user_query, with_audio=False):
        # 대화 턴 저장
        if self.last_question:
            audio_file = self.stop_recording() if with_audio else ""
            self.conversation_turns.append(ConversationTurn(
                question=self.last_question, answer=user_query,
                timestamp=datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                answer_length=len(user_query.strip()), audio_file=audio_file
            ))
        
        self.conversation_history.append({"role": "user", "content": user_query})
        self.token_count += len(self.tokenizer.encode(user_query))
        
        if self.token_count > Config.MAX_TOKENS:
            answer = "대화 시간이 다 되었어요. 수고하셨습니다."
            self.conversation_history.append({"role": "assistant", "content": answer})
            return answer, True
        
        response = self.client.chat.completions.create(
            model=Config.DEPLOYMENT, messages=self.conversation_history,
            max_tokens=1024, temperature=0.7
        )
        answer = response.choices[0].message.content
        
        self.conversation_history.append({"role": "assistant", "content": answer})
        self.token_count += len(self.tokenizer.encode(answer))
        self.last_question = answer
        
        return answer, self.token_count > Config.MAX_TOKENS

print("✅ ChatSystem 클래스 정의 완료")

# Voice System Class

In [ ]:
class VoiceSystem:
    def __init__(self):
        self.speech_config = speechsdk.SpeechConfig(subscription=Config.SPEECH_KEY, region=Config.SPEECH_REGION)
        self.speech_config.speech_recognition_language = "ko-KR"
        self.tts_voice = "ko-KR-SunHiNeural"
        Path("audio_files").mkdir(exist_ok=True)
        
        try:
            pygame.mixer.init()
            self.audio_enabled = True
        except:
            self.audio_enabled = False
    
    def transcribe_speech(self) -> str:
        try:
            recognizer = speechsdk.SpeechRecognizer(
                speech_config=self.speech_config,
                audio_config=speechsdk.audio.AudioConfig(use_default_microphone=True)
            )
            
            print("🎙️ 말씀해 주세요...")
            result = recognizer.recognize_once()
            
            if result.reason == speechsdk.ResultReason.RecognizedSpeech:
                text = result.text.strip()
                print(f"👤 \"{text}\"")
                
                exit_commands = ['종료', '그만', '끝', '나가기', 'exit', 'quit', 'stop']
                if any(cmd.lower() in text.lower().replace(' ', '') for cmd in exit_commands):
                    return "종료"
                return text
            else:
                print("❌ 음성을 인식할 수 없습니다. 다시 말씀해 주세요.")
                return ""
        except:
            return ""
    
    def synthesize_speech(self, text: str) -> str:
        if not text.strip(): return None
        
        try:
            # 토큰 요청
            token_url = f"https://{Config.SPEECH_REGION}.api.cognitive.microsoft.com/sts/v1.0/issueToken"
            token_response = requests.post(token_url, headers={"Ocp-Apim-Subscription-Key": Config.SPEECH_KEY})
            if not token_response.ok: return None
            
            # TTS 요청
            tts_url = f"https://{Config.SPEECH_REGION}.tts.speech.microsoft.com/cognitiveservices/v1"
            ssml = f"""<speak version='1.0' xml:lang='ko-KR'>
                <voice xml:lang='ko-KR' xml:gender='Female' name='{self.tts_voice}'>{text}</voice>
            </speak>"""
            
            tts_response = requests.post(tts_url, 
                headers={"Authorization": f"Bearer {token_response.text}",
                        "Content-Type": "application/ssml+xml",
                        "X-Microsoft-OutputFormat": "riff-16khz-16bit-mono-pcm"},
                data=ssml.encode("utf-8"))
            
            if tts_response.ok:
                output_path = Path("audio_files") / f"tts_{time.strftime('%Y%m%d_%H%M%S')}.wav"
                output_path.write_bytes(tts_response.content)
                
                if self.audio_enabled:
                    try:
                        pygame.mixer.music.load(str(output_path))
                        pygame.mixer.music.play()
                        while pygame.mixer.music.get_busy():
                            time.sleep(0.1)
                    except:
                        pass
                return str(output_path)
        except:
            pass
        return None

print("✅ VoiceSystem 클래스 정의 완료")

# Audio Dementia Detection_Gwona

In [ ]:
def preprocess_audio_slices(audio_path, save_path, add_noise=True):
    """로컬 wav 파일을 30초 단위로 슬라이싱하여 멜 스펙트로그램 이미지로 저장"""
    if not os.path.exists(audio_path): return []
    
    y, sr = librosa.load(audio_path, sr=SR)
    slice_length, total_slices = FIXED_DURATION * sr, len(y) // (FIXED_DURATION * sr)
    if total_slices == 0: return []
    
    os.makedirs(save_path, exist_ok=True)
    base_name = os.path.splitext(os.path.basename(audio_path))[0]
    saved_files = []
    
    for i in range(total_slices):
        y_slice = y[i * slice_length:(i + 1) * slice_length]
        
        if add_noise:
            noise = 0.005 * np.random.uniform() * np.amax(y_slice) * np.random.normal(size=y_slice.shape[0])
            y_slice = y_slice + noise
        
        mel = librosa.feature.melspectrogram(y=y_slice, sr=sr, n_mels=128)
        mel_norm = (librosa.power_to_db(mel, ref=np.max) - librosa.power_to_db(mel, ref=np.max).min()) / \
                   (librosa.power_to_db(mel, ref=np.max).max() - librosa.power_to_db(mel, ref=np.max).min())
        
        save_file = os.path.join(save_path, f"{base_name}_slice{i+1}.jpg")
        plt.figure(figsize=(10, 4))
        librosa.display.specshow(mel_norm, sr=sr, x_axis='time', y_axis='mel')
        plt.axis('off'); plt.tight_layout()
        plt.savefig(save_file, bbox_inches='tight', pad_inches=0)
        plt.close()
        saved_files.append(save_file)
    
    return saved_files

def analyze_voice_patterns(audio_path, model_path='models-05-0.7188.hdf5', save_path="./mel_slices/"):
    """음성 패턴 분석 수행"""
    saved_images = preprocess_audio_slices(audio_path, save_path, add_noise=False)
    if not saved_images:
        return {'success': False, 'message': "슬라이스된 이미지가 생성되지 않았습니다.", 'analysis': {}}
    
    try:
        model = load_model(model_path)
        predictions = []
        
        for img_path in saved_images:
            img = image.load_img(img_path, target_size=(250, 250))
            img_processed = np.expand_dims(image.img_to_array(img) / 255.0, axis=0)
            pred = model.predict(img_processed)[0]
            predictions.append(float(pred[0]) if len(pred) > 0 else float(pred))
        
        threshold, total = 0.5, len(predictions)
        positive = sum(1 for p in predictions if p >= threshold)
        ratio = positive / total if total > 0 else 0
        
        level_map = {ratio >= 0.7: ("높음", "🔴"), ratio >= 0.4: ("중간", "🟠")}.get(True, ("낮음", "🟢"))
        level, icon = level_map
        
        return {
            'success': True, 'predictions': predictions,
            'analysis': {'total_clips': total, 'positive_clips': positive, 'ratio': ratio, 'level': level, 'icon': icon}
        }
    except Exception as e:
        return {'success': False, 'message': f"분석 중 오류: {str(e)}", 'analysis': {}}

print("✅ 음성 분석 함수들 정의 완료")

#### 일단 함수 구현은 끝났는데, 리포트에 추가하는것 남음

# Story Telling / Report System

In [ ]:
class StoryGenerator:
    def __init__(self, chat_system):
        self.chat_system = chat_system
        self.client = chat_system.client
        self.strange_responses, self.rule_based_alerts = [], []
        self.conversation_id = ""
        self.voice_analysis_result = None
    
    def analyze_voice_patterns(self):
        """음성 패턴 분석 수행 (3분 이상 대화만)"""
        if not self.chat_system.session_id: return None
        
        session_audio_file = self.chat_system.session_audio_dir / f"{self.chat_system.session_id}.wav"
        if not session_audio_file.exists():
            print("⚠️ 음성 파일을 찾을 수 없어 음성 분석을 건너뜁니다.")
            return None
        
        try:
            duration = librosa.get_duration(path=str(session_audio_file))
            duration_minutes = duration / 60
            print(f"🎙️ 전체 대화 시간: {duration_minutes:.1f}분")
            
            if duration < 180:
                print(f"⚠️ 대화 시간이 {duration_minutes:.1f}분으로 3분 미만이어서 음성 분석을 건너뜁니다.")
                return {'success': False, 'duration': duration_minutes, 'reason': 'insufficient_duration'}
        except Exception as e:
            print(f"⚠️ 음성 파일 길이 확인 중 오류: {e}")
            return None
        
        print("🎙️ 음성 패턴 분석 중...")
        result = analyze_voice_patterns(str(session_audio_file), save_path=f"./mel_slices/{self.chat_system.session_id}/")
        
        if result['success']:
            result['duration'] = duration_minutes
            self.voice_analysis_result = result
            print(f"✅ 음성 분석 완료: {result['analysis']['level']} 수준 ({duration_minutes:.1f}분 대화)")
        else:
            print(f"❌ 음성 분석 실패: {result['message']}")
        return result
    
    def analyze_speech_patterns(self):
        """대화 패턴 분석"""
        if not self.chat_system.conversation_turns: return
        
        patterns = {
            'severe_depression': ["죽고싶", "살기싫", "의미없", "포기하고싶", "지쳤", "힘들어죽겠", "세상이싫", "절망"],
            'severe_anxiety': ["무서워죽겠", "불안해미쳐", "걱정돼죽겠", "두려워", "숨막혀", "공황", "패닉"],
            'severe_anger': ["화나죽겠", "미쳐버리겠", "짜증나죽겠", "열받아", "빡쳐", "분해", "참을수없"],
            'cognitive_decline': ["기억안나", "모르겠", "잊어버렸", "생각안나", "까먹었", "헷갈려", "누구였는지", "몰라"]
        }
        
        memory_issues = very_short = meaningless = 0
        repetitive = []
        
        for i, turn in enumerate(self.chat_system.conversation_turns):
            answer = turn.answer.replace(" ", "").lower()
            
            for pattern_type, keywords in patterns.items():
                for keyword in keywords:
                    if keyword in answer:
                        self.rule_based_alerts.append({
                            "type": pattern_type, "turn_number": i + 1, "keyword": keyword,
                            "answer": turn.answer, "timestamp": turn.timestamp,
                            "severity": "critical" if pattern_type == 'severe_depression' else "high"
                        })
                        if pattern_type == 'cognitive_decline': memory_issues += 1
            
            if len(turn.answer.strip()) <= 5: very_short += 1
            if turn.answer.strip() in ["음", "어", "그냥", "네", "아니", "응", "어?"]: meaningless += 1
            if i >= 3 and turn.answer.strip() in [t.answer.strip() for t in self.chat_system.conversation_turns[i-3:i]]:
                repetitive.append(i + 1)
        
        total = len(self.chat_system.conversation_turns)
        thresholds = [
            (memory_issues >= total * 0.7, "severe_memory_loss", "critical"),
            (very_short >= total * 0.8, "communication_difficulty", "high"),
            (meaningless >= total * 0.6, "cognitive_confusion", "high"),
            (len(repetitive) >= 3, "repetitive_behavior", "moderate")
        ]
        
        for condition, alert_type, severity in thresholds:
            if condition:
                self.rule_based_alerts.append({"type": alert_type, "severity": severity})
    
    def analyze_entire_conversation(self):
        """전체 대화 분석"""
        if not self.chat_system.conversation_turns: return
        
        self.strange_responses, self.rule_based_alerts = [], []
        self.analyze_speech_patterns()
        
        conversation_text = "\n".join([f"[{i}] 질문: {turn.question}\n답변: {turn.answer} (길이: {turn.answer_length}자)"
                                     for i, turn in enumerate(self.chat_system.conversation_turns, 1)])
        
        try:
            response = self.client.chat.completions.create(
                model=Config.DEPLOYMENT,
                messages=[
                    {"role": "system", "content": "치매 환자 대화 분석 전문 AI"},
                    {"role": "user", "content": f"""치매 환자 대화 분석하여 JSON 응답:
{conversation_text}

JSON: {{"conversation_analysis": [{{"turn_number": 1, "is_strange": true/false, "severity": "normal/mild/moderate/severe", "emotion": "감정", "answer_quality": "poor/normal/good/excellent", "reason": "이유"}}], "overall_assessment": {{"dominant_emotion": "주요감정", "cognitive_level": "normal/mild_concern/moderate_concern/severe_concern"}}}}

감정: 기쁨,슬픔,그리움,무력감,우울감,분노,불안,중립,감사,애정,흥미,짜증"""}
                ], max_tokens=1024, temperature=0.1
            )
            
            text = response.choices[0].message.content
            if "```json" in text:
                text = text[text.find("```json")+7:text.find("```", text.find("```json")+7)]
            elif "{" in text:
                text = text[text.find("{"):text.rfind("}")+1]
            
            result = json.loads(text)
            
            for i, analysis in enumerate(result.get("conversation_analysis", [])):
                if i < len(self.chat_system.conversation_turns):
                    turn = self.chat_system.conversation_turns[i]
                    turn.emotion = analysis.get("emotion", "중립")
                    turn.answer_quality = analysis.get("answer_quality", "normal")
                    
                    if analysis.get("is_strange", False):
                        self.strange_responses.append(StrangeResponse(
                            question=turn.question, answer=turn.answer, timestamp=turn.timestamp,
                            severity=analysis.get("severity", "mild"), emotion=turn.emotion,
                            answer_quality=turn.answer_quality
                        ))
            return result
        except:
            return None
    
    def calculate_ratings(self):
        """평점 계산"""
        total = len(self.chat_system.conversation_turns)
        if total == 0: return {"emotion": 3, "coherence": 3, "overall": 3}
        
        emotions = [turn.emotion for turn in self.chat_system.conversation_turns if hasattr(turn, 'emotion')]
        emotion_counts = {}
        for emotion in emotions: emotion_counts[emotion] = emotion_counts.get(emotion, 0) + 1
        
        positive = ["기쁨", "그리움", "감사", "애정", "흥미"]
        negative = ["슬픔", "무력감", "우울감", "분노", "불안", "짜증"]
        pos_count = sum(emotion_counts.get(e, 0) for e in positive)
        neg_count = sum(emotion_counts.get(e, 0) for e in negative)
        
        # 감정 평점
        critical_alerts = [a for a in self.rule_based_alerts if a.get('severity') == 'critical']
        if critical_alerts: emotion_rating = 1
        elif neg_count > pos_count * 2: emotion_rating = 2
        elif neg_count > pos_count: emotion_rating = 3
        elif pos_count > neg_count: emotion_rating = 4
        else: emotion_rating = 5 if pos_count > neg_count * 2 else 3
        
        # 일관성 평점
        strange_pct = len(self.strange_responses) / total * 100
        severe_count = sum(1 for r in self.strange_responses if r.severity == 'severe')
        if strange_pct == 0: coherence_rating = 5
        elif strange_pct <= 20 and severe_count == 0: coherence_rating = 4
        elif strange_pct <= 40 and severe_count <= 1: coherence_rating = 3
        elif strange_pct <= 60 or severe_count <= 2: coherence_rating = 2
        else: coherence_rating = 1
        
        # 전체 평점
        qualities = [turn.answer_quality for turn in self.chat_system.conversation_turns if hasattr(turn, 'answer_quality')]
        quality_counts = {"poor": 0, "normal": 0, "good": 0, "excellent": 0}
        for q in qualities: quality_counts[q] += 1
        
        poor_pct = quality_counts["poor"] / total * 100
        if critical_alerts or poor_pct >= 50: overall_rating = 1
        elif poor_pct >= 30: overall_rating = 2
        elif quality_counts["excellent"] / total >= 0.3: overall_rating = 5
        elif quality_counts["good"] / total >= 0.3: overall_rating = 4
        else: overall_rating = 3
        
        return {"emotion": emotion_rating, "coherence": coherence_rating, "overall": overall_rating}
    
    def format_star_rating(self, rating):
        return f"{'⭐' * rating}{'☆' * (5 - rating)} ({rating}/5)"
    
    def generate_report(self):
        """리포트 생성 (간소화)"""
        # 음성 분석 수행
        if hasattr(self.chat_system, 'session_id') and self.chat_system.session_id:
            self.analyze_voice_patterns()
        
        self.analyze_entire_conversation()
        total = len(self.chat_system.conversation_turns)
        if total == 0: return "대화가 진행되지 않았습니다."
        
        # 감정 분석
        emotions = [turn.emotion for turn in self.chat_system.conversation_turns if hasattr(turn, 'emotion')]
        emotion_counts = {}
        for emotion in emotions: emotion_counts[emotion] = emotion_counts.get(emotion, 0) + 1
        
        dominant_emotion = max(emotion_counts, key=emotion_counts.get) if emotion_counts else "중립"
        ratings = self.calculate_ratings()
        critical_alerts = [a for a in self.rule_based_alerts if a.get('severity') == 'critical']
        
        # 리포트 생성
        report = f"""
{'='*60}
📋 치매 진단 대화 분석 리포트
{'='*60}
📅 분석 일시: {datetime.now().strftime('%Y년 %m월 %d일 %H:%M:%S')}
🆔 대화 ID: {self.conversation_id}
{'='*60}

🎯 종합 평가
{'─'*30}
😊 감정 상태:     {self.format_star_rating(ratings['emotion'])}
💬 답변 일관성:   {self.format_star_rating(ratings['coherence'])}
🧠 전반적 인지:   {self.format_star_rating(ratings['overall'])}"""

        # 음성 분석 결과 추가
        if self.voice_analysis_result:
            if self.voice_analysis_result.get('success'):
                va = self.voice_analysis_result['analysis']
                duration = self.voice_analysis_result.get('duration', 0)
                report += f"\n🎙️ 음성 패턴:     {va['icon']} {va['level']} ({va['ratio']:.0%}, {duration:.1f}분)"
            else:
                duration = self.voice_analysis_result.get('duration', 0)
                report += f"\n🎙️ 음성 패턴:     ⚪ 분석 불가 (대화시간 {duration:.1f}분 < 3분)"
        
        report += f"\n{'─'*30}\n\n📊 대화 개요\n{'─'*30}"
        report += f"\n💬 총 대화 횟수: {total}회"
        report += f"\n😊 전반적 감정: {dominant_emotion}"
        report += f"\n{'✅ 어긋난 답변: 없음' if len(self.strange_responses) == 0 else f'⚠️ 어긋난 답변: {len(self.strange_responses)}회'}"
        report += f"\n{'✅ 발화 패턴: 특이사항 없음' if len(self.rule_based_alerts) == 0 else f'🔍 발화 패턴: {len(self.rule_based_alerts)}건 관찰'}"
        
        # 음성 분석 개요
        if self.voice_analysis_result and self.voice_analysis_result.get('success'):
            va = self.voice_analysis_result['analysis']
            duration = self.voice_analysis_result.get('duration', 0)
            report += f"\n🎙️ 음성 분석: {va['total_clips']}개 클립 중 {va['positive_clips']}개에서 치매 가능성 감지 ({duration:.1f}분)"
        
        report += f"\n{'─'*30}\n\n"
        
        # 음성 분석 상세 결과
        if self.voice_analysis_result:
            if self.voice_analysis_result.get('success'):
                va = self.voice_analysis_result['analysis']
                duration = self.voice_analysis_result.get('duration', 0)
                report += f"""🎙️ 음성 기반 치매 예측 결과
{'─'*30}
⏱️ 전체 대화 시간: {duration:.1f}분
🧪 전체 분석 클립: {va['total_clips']}개
🧠 치매 가능성으로 분류된 클ip: {va['positive_clips']}개
📊 예측 비율: {va['ratio']:.0%}
{va['icon']} 치매 가능성 수준: {va['level']}
📌 참고: 이 결과는 음성의 억양, 피치, 떨림 등 음향적 특성을 기반으로 합니다.
{'─'*30}

"""
            elif self.voice_analysis_result.get('reason') == 'insufficient_duration':
                duration = self.voice_analysis_result.get('duration', 0)
                report += f"""🎙️ 음성 분석 안내
{'─'*30}
⏱️ 전체 대화 시간: {duration:.1f}분
⚠️ 정확한 음성 패턴 분석을 위해서는 최소 3분 이상의 대화가 필요합니다.
💡 다음번에는 조금 더 길게 대화해보시면 음성 분석 결과도 함께 확인하실 수 있습니다.
{'─'*30}

"""
        
        # 권장사항
        voice_warning = 0
        if self.voice_analysis_result and self.voice_analysis_result.get('success'):
            level = self.voice_analysis_result['analysis']['level']
            voice_warning = 2 if level == "높음" else 1 if level == "중간" else 0
        
        report += f"💡 권장사항\n{'─'*30}\n"
        if critical_alerts:
            report += "🚨 긴급 권장사항: 심각한 정신건강 위험 신호가 감지되었습니다.\n"
        elif len([a for a in self.rule_based_alerts if a.get('severity') == 'high']) >= 2 or voice_warning >= 2:
            report += "⚠️ 주의 권장사항: 최근 대화에서 혼란스러운 답변이 자주 보였습니다.\n"
        elif len(self.strange_responses) > 0:
            report += "💙 관심 권장사항: 전반적으로 잘 응답해주셨지만, 간혹 어긋난 답변이 보입니다.\n"
        else:
            report += "💚 훌륭한 상태: 어르신께서 무척 안정적으로 잘 응답해주셨습니다.\n"
        
        report += f"{'─'*30}\n\n{'='*60}\n📋 리포트 끝 - 어르신의 건강과 행복을 위해\n{'='*60}"
        
        return report
    
    def save_files(self, image_path):
        """파일 저장"""
        # 폴더 구조 생성
        image_basename = os.path.splitext(os.path.basename(image_path))[0]
        image_dir = Path("conversation_log") / image_basename
        image_dir.mkdir(parents=True, exist_ok=True)
        
        existing_dirs = list(image_dir.glob(f"{image_basename}_conv*"))
        conv_number = len(existing_dirs) + 1
        self.conversation_id = f"{image_basename}_conv{conv_number}"
        
        conversation_dir = image_dir / self.conversation_id
        conversation_dir.mkdir(exist_ok=True)
        
        # 개별 QA 저장
        for i, turn in enumerate(self.chat_system.conversation_turns, 1):
            qa_file = conversation_dir / f"qa_{i:02d}.txt"
            qa_file.write_text(f"""=== 질의응답 {i}번 ===
대화 ID: {self.conversation_id}
시간: {turn.timestamp}
{'='*25}

🤖 질문:
{turn.question}

👤 답변:
{turn.answer}
{'='*25}""", encoding='utf-8')
        
        # 메인 대화 파일
        conversation_file = conversation_dir / f"{self.conversation_id}.txt"
        conversation_content = f"""{'='*50}
💬 치매 진단 대화 기록
{'='*50}
🆔 대화 ID: {self.conversation_id}
📊 총 대화 수: {len(self.chat_system.conversation_turns)}회
{'='*50}

"""
        for turn in self.chat_system.conversation_turns:
            conversation_content += f"[{turn.timestamp}]\n🤖 질문: {turn.question}\n👤 답변: {turn.answer}\n{'-'*30}\n\n"
        
        conversation_file.write_text(conversation_content, encoding='utf-8')
        
        # 분석 리포트 저장
        analysis_dir = Path("analysis")
        analysis_dir.mkdir(exist_ok=True)
        analysis_file = analysis_dir / f"{self.conversation_id}_analysis.txt"
        analysis_file.write_text(self.generate_report(), encoding='utf-8')
        
        print(f"✅ 파일 저장 완료!\n📁 대화 폴더: {conversation_dir}\n📄 대화 파일: {conversation_file}\n📊 분석 파일: {analysis_file}")
        return str(conversation_file), str(analysis_file)
    
    def generate_story(self, image_path):
        """추억 스토리 생성"""
        conversation_text = "\n".join([f"질문: {turn.question}\n답변: {turn.answer}" for turn in self.chat_system.conversation_turns])
        if not conversation_text.strip(): return None, None
        
        try:
            response = self.client.chat.completions.create(
                model=Config.DEPLOYMENT,
                messages=[
                    {"role": "system", "content": "노인 추억 스토리텔러"},
                    {"role": "user", "content": f"""대화 기반으로 어르신 1인칭 추억 스토리 15줄 작성:
{conversation_text}
지침: 답변 기반 작성, 감정과 감각 포함, 따뜻한 톤, 손자/손녀에게 들려주는 어투"""}
                ], max_tokens=512, temperature=0.8
            )
            
            story = response.choices[0].message.content
            story_dir = Path("story_telling")
            story_dir.mkdir(exist_ok=True)
            
            story_file = story_dir / f"{os.path.splitext(os.path.basename(image_path))[0]}_story.txt"
            story_file.write_text(story, encoding='utf-8')
            
            return story, str(story_file)
        except:
            return None, None

print("✅ StoryGenerator 클래스 정의 완료")

# 코드 통합부

In [ ]:
class OptimizedDementiaSystem:
    def __init__(self):
        self.image_analyzer = ImageAnalyzer()
        self.chat_system = ChatSystem()
        self.voice_system = VoiceSystem() if Config.SPEECH_KEY else None
        self.story_generator = StoryGenerator(self.chat_system)
    
    def start_conversation(self, image_path):
        """이미지 분석 및 대화 시작"""
        if not os.path.exists(image_path): return None
        
        analysis_result = self.image_analyzer.analyze_image(image_path)
        if not analysis_result: return None
        
        self.chat_system.setup_conversation_context(analysis_result)
        return self.chat_system.generate_initial_question()
    
    def run_conversation(self, image_path, is_voice=False):
        """대화 실행"""
        initial_question = self.start_conversation(image_path)
        if not initial_question: return None
        
        # 시작 메시지
        if is_voice and self.voice_system:
            welcome = "안녕하세요. 사진을 보며 대화해요."
            print(f"🤖 {welcome}")
            self.voice_system.synthesize_speech(welcome)
            print(f"🤖 {initial_question}")
            self.voice_system.synthesize_speech(initial_question)
        else:
            print(f"🤖 {initial_question}")
        
        conversation_type = "음성" if is_voice else "텍스트"
        print(f"\n{'='*40}\n{'🎙️' if is_voice else '💬'} {conversation_type} 대화 시작!")
        print(f"💡 {'종료라고 말하면' if is_voice else 'exit 또는 종료를 입력하면'} 끝납니다\n{'='*40}")
        
        # 대화 루프
        while True:
            if is_voice and self.voice_system:
                print("🎙️ 말씀해 주세요...")
                self.chat_system.start_recording()
                user_input = self.voice_system.transcribe_speech()
                self.chat_system.stop_recording()
                
                if not user_input.strip(): continue
                if user_input == "종료":
                    end_msg = "대화를 마치겠습니다. 감사합니다."
                    print(f"🤖 {end_msg}")
                    self.voice_system.synthesize_speech(end_msg)
                    break
            else:
                user_input = input("\n👤 답변: ").strip()
                if user_input.lower() in ['exit', '종료', 'quit', 'q']:
                    print("대화를 종료합니다.")
                    break
            
            answer, should_end = self.chat_system.chat_about_image(user_input, with_audio=is_voice)
            print(f"🤖 {answer}")
            
            if is_voice and self.voice_system:
                self.voice_system.synthesize_speech(answer)
            
            if should_end:
                end_msg = "대화 시간이 종료되었습니다."
                print(f"⏰ {end_msg}")
                if is_voice and self.voice_system:
                    self.voice_system.synthesize_speech(end_msg)
                break
        
        # 분석 및 저장
        print("\n📊 종합 분석 결과 생성 중...")
        conversation_file, analysis_file = self.story_generator.save_files(image_path)
        story, story_file = self.story_generator.generate_story(image_path)
        
        print(self.story_generator.generate_report())
        
        if story:
            print(f"\n{'='*50}\n📖 생성된 추억 이야기\n{'='*50}\n{story}\n{'='*50}")
        
        print(f"📂 대화기록: {conversation_file}\n📊 분석결과: {analysis_file}")
        if story_file: print(f"📖 스토리: {story_file}")
        
        return {
            'conversation_file': conversation_file, 'analysis_file': analysis_file,
            'story_file': story_file, 'story_content': story,
            'conversation_id': self.story_generator.conversation_id
        }

print("✅ OptimizedDementiaSystem 클래스 정의 완료")

In [ ]:
def interactive_conversation():
    """텍스트 대화 실행"""
    print("=== 💬 텍스트 치매 진단 대화 시스템 ===")
    image_path = "images.jpg"
    
    if not os.path.exists(image_path):
        print("❌ 올바른 이미지 경로를 입력해주세요.")
        return None
    
    try:
        system = OptimizedDementiaSystem()
        return system.run_conversation(image_path, is_voice=False)
    except Exception as e:
        print(f"❌ 시스템 오류: {e}")
        return None

def interactive_voice_conversation():
    """음성 대화 실행"""
    print("=== 🎤 음성 치매 진단 대화 시스템 ===")
    image_path = input("이미지 경로를 입력하세요: ").strip()
    
    if not image_path or not os.path.exists(image_path):
        print("❌ 올바른 이미지 경로를 입력해주세요.")
        return None
    
    try:
        system = OptimizedDementiaSystem()
        if not system.voice_system:
            print("❌ 음성 시스템을 초기화할 수 없습니다. Azure Speech Service 키를 확인해주세요.")
            return None
        return system.run_conversation(image_path, is_voice=True)
    except Exception as e:
        print(f"❌ 시스템 오류: {e}")
        return None

print("✅ 실행 함수들 정의 완료")

In [ ]:
if __name__ == "__main__":
    # 환경 확인
    if not Config.ENDPOINT or not Config.SUBSCRIPTION_KEY:
        print("⚠️ Azure OpenAI 설정이 필요합니다:")
        print("   - gpt-endpoint")
        print("   - gpt-key")
    
    if not Config.SPEECH_KEY:
        print("⚠️ 음성 기능을 위해 Azure Speech Service 설정이 필요합니다:")
        print("   - speech-key")
    
    print("\n✅ 시스템 준비 완료!")
    print("💡 interactive_conversation() 함수를 실행해보세요!")


✅ 시스템 준비 완료!
💡 interactive_conversation() 함수를 실행해보세요!


In [ ]:
interactive_voice_conversation()

=== 🎤 음성 치매 진단 대화 시스템 ===
🤖 안녕하세요. 사진을 보며 대화해요.
🤖 "어르신, 이 사진 속 축하 행사 분위기가 참 따뜻하네요. 예전에 이런 모임에 참석해보신 적 있으신가요?"

🎙️ 음성 대화 시작!
💡 종료라고 말하면 끝납니다
🎙️ 말씀해 주세요...
🎙️ 말씀해 주세요...
👤 "에 참석해온 적 있어?"
🤖 "아, 그러셨군요! 축하 모임은 늘 특별하죠. 혹시 기억에 남는 축하 행사나 분위기가 떠오르시나요?"
🎙️ 말씀해 주세요...
🎙️ 말씀해 주세요...
👤 "어 기억에 남는건데 생일 생일날도 있어?"
🤖 "생일날이라니, 정말 특별한 날이셨겠어요! 생일날에 어떤 분위기였는지 더 자세히 들려주실 수 있을까요?"
🎙️ 말씀해 주세요...
🎙️ 말씀해 주세요...
👤 "세미나를 케이크 먹었지?"
🤖 "세미나에서 케이크를 드셨다니 재미있네요! 세미나 분위기는 어땠나요? 사람들도 많이 모였었나요?"
🎙️ 말씀해 주세요...
🎙️ 말씀해 주세요...
👤 "종료."
🤖 대화를 마치겠습니다. 감사합니다.

📊 종합 분석 결과 생성 중...
📁 저장 구조:
   메인 폴더: conversation_log/images/images_conv5/
   대화 파일: images_conv5.txt

✅ 파일 저장 완료!
📁 대화 폴더: conversation_log\images\images_conv5
📄 대화 파일: conversation_log\images\images_conv5\images_conv5.txt
📊 분석 파일: analysis\images_conv5_analysis.txt
📋 QA 파일들: 3개

📋 치매 진단 대화 분석 리포트
📅 분석 일시: 2025년 06월 09일 10:40:56
🆔 대화 ID: images_conv5

🎯 종합 평가
──────────────────────────────
😊 감정 상태:     ⭐⭐⭐⭐☆ (4/5)
💬 답변 일관성:   ⭐⭐☆☆☆ (2/5)
🧠 전반적 인지:   ⭐☆☆☆☆ (1/5)
───────────

{'conversation_file': 'conversation_log\\images\\images_conv5\\images_conv5.txt',
 'analysis_file': 'analysis\\images_conv5_analysis.txt',
 'story_file': 'story_telling\\images_story.txt',
 'story_content': '"아, 그렇구나. 이 사진을 보니 참 따뜻한 축하 분위기가 느껴지네. 나도 이런 축하 행사에 여러 번 참석했던 기억이 있어. 정말 특별한 순간들이었지. 특히 기억에 남는 건 내 생일날이었어. 오래전 일인데도 생생하게 떠오른다.\n\n그날은 내가 조금 더 특별한 사람이라도 된 것처럼 모두가 나를 중심으로 모였지. 손님들은 웃음 가득한 얼굴로 축하해주고, 카페에선 생일 케이크가 준비되어 있었어. 그 케이크는 정말로 크고 아름다웠는데, 위에 초도 꽂혀 있었지. 초를 불었을 때 사람들이 박수를 치며 환호하던 소리가 아직도 귀에 선하구나.\n\n가장 기억에 남는 건 케이크를 나눠 먹던 순간이었어. 한 입 베어 문 초콜릿 케이크의 달콤함과 크림의 부드러움이 정말 황홀했지. 그때는 지금처럼 사진 찍는 문화가 없었으니, 마음으로 그 순간을 간직했어. 사람들의 따뜻한 웃음과 내가 느꼈던 행복은 정말 소중한 추억이야.\n\n내 친구들 중 한 명은 기타를 가져와서 축하 노래를 불러줬는데, 정말 감동적이었어. 그 멜로디가 바람을 타고 퍼져나가며 그 날을 더욱 특별하게 만들었지. 모두가 함께 박수를 치며 노래를 따라 부르던 그 순간은 말로 다 표현할 수 없을 만큼 즐거웠어.\n\n그리고 생일 선물도 빼놓을 수 없지. 작은 상자에 담긴 손수건을 받았는데, 그 손수건은 얼마나 귀했던지 지금도 간직하고 있단다. 손으로 꾹꾹 눌러서 내 이름을 수놓은 손수건이었어. 그 마음이 너무나 고맙고 사랑스러웠지.\n\n그날의 따뜻한 기억은 내 삶에서 빛나는 보석 같은 날들이야. 손주야, 너도 그런 행복한 순간을 많이 만들어가면 좋겠어.